<a href="https://colab.research.google.com/github/ShreyJais/LLM/blob/main/Shreyansh_558_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#dataset

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-439lyv6l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-439lyv6l
  Resolved https://github.com/huggingface/transformers.git to commit 5b7a225f2502840f4066b1b39ef53411ea4d6e90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9050429 sha256=8b1d36113c53f69f644939dd76f5ff0f7a23999b3272d0137084fcb4f8ed52ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrph4ado/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-

This code installs three libraries you'll need to work with large language models:

* transformers: This library provides tools to download and use pre-trained language models.
* datasets: This library helps you find and download datasets to train or use with your language models.
* huggingface-hub: This library allows you to interact with the Hugging Face Hub, a platform for sharing and accessing language models and datasets.

In [ ]:
from datasets import load_dataset
datasets = load_dataset("its-myrto/fitness-question-answers")    '''a publicly available dataset created by Rajpurkar
                                        and it's related to a task called SQUAD (Stanford Question Answering Dataset).'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
Train = pd.DataFrame(datasets['train'])
Train

In [ ]:
Val = pd.DataFrame(datasets['validation'])
Val

pd.DataFrame(datasets['train/validation']) : This converts the selected training/validation data (likely a dictionary or list) into a Pandas DataFrame for easier handling.

#Preprocessing the training data

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

This code snippet is written in Python and uses the Transformers library for natural language processing (NLP) tasks. Let's break it down:

> from transformers import AutoTokenizer: This line imports the **AutoTokenizer class** from the Transformers library. This class helps **convert text into a format that machine learning models can understand.**

> model_checkpoint = "distilbert-base-cased": This line defines a variable named model_checkpoint and sets its value to "distilbert-base-cased". This **refers to a *specific pre-trained model* from the Transformers library called "distilbert-base-cased"**. Pre-trained models are already trained on a massive amount of text data and can be fine-tuned for various NLP tasks.

> tokenizer = AutoTokenizer.from_pretrained(model_checkpoint): This line **uses the AutoTokenizer class to load the tokenizer specifically designed for the chosen pre-trained model ("distilbert-base-cased")**. The loaded tokenizer will be stored in the tokenizer variable.

In [ ]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)


 Example:

Original text: "The quick brown fox jumps over the lazy dog. It was a sunny day."

Chunk 1: "The quick brown fox jumps over the lazy dog."

Chunk 2 (with doc_stride of 3): "fox jumps over the lazy dog. It was a sunny day."

Explanation: "doc_stride" controls the number of overlapping tokens between consecutive chunks to ensure context continuity.



In [ ]:
def prepare_train_features(examples):

    #Cleaning Data: Removes any extra spaces at the beginning of each question and context.

    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]

    tokenized_examples = tokenizer(

        # 1. The question and context after removing leading and trailing spaces from the text
        examples["question"],
        examples["context"],

        # 2. Truncation: If the second sequence(here-context) exceed max_length(here=384 characters), we truncates (split it off) to make sure it fits within 384 characters.
        truncation="only_second",
        max_length=max_length,

        # 3. Stride: If the text is too long and gets split, we use a sliding window to overlap some parts(i.e., after splitting into chunks of text, last few words from the current chunk is repeated in the consecutive chunk), so we don’t miss any important information.
                    #[A sliding window breaks down long texts into smaller, manageable, overlapping parts, helping to process or analyze the text while maintaining the context.]
        stride=doc_stride,

        # 4. Overflowing Tokens: We keep those extra tokens (the tokens of the text(after tokenisation) which exceeds the maximum limit provided).
        return_overflowing_tokens=True,

        # 5. Offset Mapping: We keep track of where each token came from in the original tokenized text.
                #[Original Tokenized Text: The initial breakdown of the text into tokens before any further processing like truncation or padding.]
                #[For example, if the model identifies the answer in the tokenized context, the offsets help us find the corresponding position in the original context.]
        return_offsets_mapping=True,

        # 6. We make sure all the tokenized texts are the same length by adding padding (extra tokens) if needed.
        padding="max_length",

    )



In [ ]:
EXPLANATION OF THIS CODE BLOCK IN DETAIL IS GIVEN BELOW:
    # 1. Retrieve/take out/pop Mappings:

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")       #helps us know that all chunks come from the same original text.
    offset_mapping = tokenized_examples.pop("offset_mapping")                   #helps us know that all chunks come from the same original text.

    # 2. Prepare for Start and End Positions:

    tokenized_examples["start_positions"] = []                        #We create empty lists to store the start and end positions of the 'answers within the tokenized text'.
    tokenized_examples["end_positions"] = []

    # 3. Iterate Over Each Example and Extract Relevant Data:

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]                #The token IDs for the current tokenized example.
        cls_index = input_ids.index(tokenizer.cls_token_id)           #finds the index of the classification token (usually denoted by [CLS]) within the input_ids list.
                                                                                #This token is often used to indicate the beginning of a sentence or the entire input sequence(question+context).

        sequence_ids = tokenized_examples.sequence_ids(i)             #Identifies which part of the input this current token belongs to (e.g., question or context).

        sample_index = sample_mapping[i]                              # retrieves the index of the original example(context/question) this tokenized example corresponds to, using the "overflow_to_sample_mapping" we popped earlier
        answers = examples["answers"][sample_index]                   # retrieves the answer information from the original example(context) using the index and the "answers" key from the separate examples dictionary(the dictinary which contains answers for the corresponding questions).
                                                                                #This new dictionary likely contains answer text and its starting character "position" of the answer word from the original context
     # 4. Handle No Answer and Answer Case

        if len(answers["answer_start"]) == 0:                         #If there are no answers provided, we use the [CLS] token as the answer position-to consider the entire input sequence from beginning to its end
                                                                                #because the model doesn't have an explicit answer to reference in this case.
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)


        else:                                                         #Otherwise, we find the start and end character positions of the answer in the original context/passage.

            start_char = answers["answer_start"][0]                   #retrieves the starting character position of the answer within the original context (passage) using the "answer_start" key in the "answers" dictionary.
                                                                                #This is typically the 0th element in the list since it likely stores a single answer for each question.The value 0 in this list signifies that the answer starts at the very beginning (character position 0) of the first sentence in the context.
            end_char = start_char + len(answers["text"][0])


     #  5. Find Token Start and End Indices to search for the answer within the context part only [Remember: sequence_ids=0 means question ; 1 means context]

            token_start_index = 0                                     #This variable will keep track of the starting token index of the answer within the tokenized sequence.
            while sequence_ids[token_start_index] != 1:               #The while loop iterates through the sequence_ids list for the current example.
                token_start_index += 1                                #This list uses special tokens (0 for question, 1 for context) to separate the question and answer parts within the combined sequence.
                                                                                #ensures we start searching for the answer within the context portion of the tokenized sequence.


            token_end_index = len(input_ids) - 1                      #This variable will keep track of the ending token index of the answer within the tokenized sequence.
            while sequence_ids[token_end_index] != 1:                 #The while loop iterates backwards through the sequence_ids list for the current example.
                token_end_index -= 1                                  #initialized to the last element's index in the input_ids list (length minus 1).
                                                                                #This variable will keep track of the ending token index of the answer within the tokenized sequence.
                                                                                #It keeps decrementing token_end_index until it finds the last token that belongs to the context part (marked by 1 in sequence_ids).
                                                                                #ensures that we end the search for the answer within the context portion.


      # 6. Check if answer is Within the identified context Span in the tokenised sequence ELSE Adjust Start and End Indices to Fit Answer

            if not (
                offsets[token_start_index][0] <= start_char           #Is the starting character of the answer in the tokenized sequence less than or equal to the original answer's starting character position (start_char)?
                and offsets[token_end_index][1] >= end_char           #Is the ending character of the answer in the tokenized sequence greater than or equal to the original answer's ending character position (end_char)?
            ):
                tokenized_examples["start_positions"].append(cls_index)         #If both these conditions are true, it means the answer found within the tokenized sequence
                tokenized_examples["end_positions"].append(cls_index)                     #(between token_start_index and token_end_index) aligns with the original answer in terms of character positions.
                                                                                          #This is the ideal scenario.


            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

      # 7. Return Result: Return the modified tokenized_examples with the added start and end positions for the answers.

    return tokenized_examples

## 1. **Sampling mapping**
* The "overflow_to_sample_mapping" is a key in the tokenized_examples dictionary that helps ***map each tokenized chunk back to its original sample in the dataset***. This is especially important when dealing with long texts that have been split into multiple smaller chunks due to size constraints.

  ### **Explanation with Example:**

Let's break it down with an example for better understanding:

> **Original Data**

> Question: "What is the capital of France?"

> Context: "France is a country in Europe. Its capital is Paris, known for its culture and landmarks."

* Tokenization and Chunking:

When the context is too long, it gets split into smaller chunks. For instance:

> Chunk 1: ["What", "is", "the", "capital", "of", "France", "[CLS]", "France", "is", "a", "country"]

> Chunk 2: ["France", "is", "a", "country", "in", "Europe", ".", "[CLS]", "Its", "capital", "is"]

> Chunk 3: ["Its", "capital", "is", "Paris", ",", "known", "for", "its", "culture", "and", "landmarks", "."]

  * sample_mapping = [0, 0, 0]


> Explanation :

  * Value: Each element in the sample_mapping list corresponds to a chunk and indicates which original sample it came from.

  * 0 means all three chunks are from the first original sample (Sample 1).


## 2. **Offset mapping**
When text is tokenized (split into smaller pieces like words or subwords), each token retains ***information about where it came from in the original text***. This is important for understanding how the tokenized text maps back to the original text.

  ### **Example:**
* For the sentence "The quick brown fox":

> Tokens might be: ["The", "quick", "brown", "fox"]

> Offsets might be: [(0, 3), (4, 9), (10, 15), (16, 19)]

> Each pair of numbers in the offsets tells you where each token starts and ends in the original sentence.

Let's break down the code you provided step-by-step and explain it in simple terms:

### Initial Setup

1. **Retrieve Mappings**: We take out (pop) the "overflow_to_sample_mapping" and "offset_mapping" from `tokenized_examples`.
   - `sample_mapping` tells us which original sample each tokenized example corresponds to.
   - `offset_mapping` provides the start and end positions of each token in the original text.


2. **Prepare for Start and End Positions**: We create empty lists to store the start and end positions of the answers within the tokenized text.

### Main Loop

3. **Iterate Over Each Example**: Loop through each tokenized example and its corresponding offsets.

4. **Extract Relevant Data**:
   - `input_ids`: The token IDs for the current tokenized example.
   - `cls_index`: The index of the `[CLS]` token, which is a special token used by the tokenizer.
   - `sequence_ids`: Identifies which part of the input each token belongs to (e.g., question or context).
   - `sample_index`: The index of the original sample in the dataset.
   - `answers`: The answer(s) for the current sample.

### Handle No Answer Case

5. **No Answer**: If there are no answers provided, we use the `[CLS]` token as the answer position.

### Handle Answer Case

6. **Get Answer Positions**: Find the start and end character positions of the answer in the original text.

### Find Token Start and End Indices

7. **Locate Answer in Tokens**:
   - `token_start_index`: Find the start of the context in the token list.
   - `token_end_index`: Find the end of the context in the token list.

### Check If Answer is Within the Span

8. **Answer Out of Span**: If the answer isn't fully within the current chunk of tokens, use the `[CLS]` token as the position.

### Adjust Start and End Indices to Fit Answer

9. **Adjust Indices**: Move the `token_start_index` and `token_end_index` to exactly match the start and end of the answer. Add these positions to the lists.

### Final Return

10. **Return Result**: After processing all examples, return the modified `tokenized_examples` with the added start and end positions for the answers.

### Summary

- This code processes tokenized text to map answer positions from the original text to the tokenized version.
- It handles cases where answers span across token chunks or are missing.
- The final output includes the positions of answers in tokenized form, ready for training a model like BERT for question answering tasks.

In [ ]:
tokenized_datasets = datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets["train"].column_names,
    num_proc=3,
)

Map (num_proc=3):   0%|          | 0/87599 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
train_set = tokenized_datasets["train"].with_format("numpy")[:]  # Load the whole dataset as a dict of numpy arrays
validation_set = tokenized_datasets["validation"].with_format("numpy")[:]

## FINE TUNING THE MODEL

In [ ]:
from transformers import TFAutoModelForQuestionAnswering                  #used for question answering tasks.

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [ ]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
# Optionally uncomment the next line for float16 training
keras.mixed_precision.set_global_policy("mixed_float16")

model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x792c086316c0>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x792c086316c0>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2773/2773 [==============================] - 3876s 1s/step - loss: 1.5155 - val_loss: 1.1785


In [ ]:
context = """Natural language processing (NLP) is a machine learning technology that gives computers the ability to interpret, manipulate, and comprehend human language.
Organizations today have large volumes of voice and text data from various communication channels like emails, text messages, social media newsfeeds, video, audio, and more.
They use NLP software to automatically process this data, analyze the intent or sentiment in the message, and respond in real time to human communication.
"""
question = "what is NLP?"

inputs = tokenizer([context], [question], return_tensors="np")      #The context and question are tokenized using a tokenizer. It returns input tensors.
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)            #The model processes the inputs, which generate start and end logits for the answer span.
                                                                    #The code then identifies the start and end positions of the answer by finding the argmax of the start and end logits.
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))

9 88


What are logits?

In the context of neural networks, **logits are the vector of raw (non-normalized) predictions that a model generates.**

Logits are the result of applying a linear transformation to the last hidden layer of a neural network.

**In the case of question answering, the model uses logits to represent the confidence scores for each token in the input sequence being the start or end of the answer span.**

* Generating start and end logits:

When the input data (context and question) is passed through the pre-trained question answering model, the model generates start and end logits.

These logits represent the model's confidence regarding each token's potential to be the start or end of the answer within the given context.

* Identifying the answer span:

Once the start and end logits are generated, *the next step is to identify the start and end positions of the answer within the input sequence.* - *achieved by taking the argmax (the index of the maximum value) of the start logits and end logits independently*.

This gives us the token indices that the model considers the most likely start and end positions for the answer span within the provided context.

* Extracting the answer span:

With the identified start and end positions, the code extracts the corresponding answer span from the input sequence. This means that the tokens between the identified start and end positions in the input sequence are considered as the predicted answer span.

In [ ]:
answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]      #Finally, the answer is decoded using the tokenizer.
print(answer)


[  170  3395  3776  2815  1115  3114  7565  1103  2912  1106 19348   117
 19109   117  1105 24822  1769  1846   119 25684  2052  1138  1415  6357
  1104  1490  1105  3087  2233  1121  1672  4909  6412  1176 24853   117
  3087  7416   117  1934  2394  2371  8124 15513   117  1888   117  6056
   117  1105  1167   119  1220  1329 21239  2101  3594  1106  7743  1965
  1142  2233   117 19774  1103  7676  1137 17024  1107  1103  3802   117
  1105  6297  1107  1842  1159  1106  1769  4909]


In [ ]:
print(tokenizer.decode(answer))

a machine learning technology that gives computers the ability to interpret, manipulate, and comprehend human language. Organizations today have large volumes of voice and text data from various communication channels like emails, text messages, social media newsfeeds, video, audio, and more. They use NLP software to automatically process this data, analyze the intent or sentiment in the message, and respond in real time to human communication
